# BE m228 Final Project

### Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import shap

### Data Processing

**Gut Health Data**

In [ ]:
microbiome = pd.read_csv('data/gut_health_test.csv')
microbiome.dropna(inplace=True)

microbiome.head(3)

I dropped any rows with missing values which leaves us only with 42 participants. 

- subjects with missing vals: 26, 28, 48
- already dropped: 24, 25, 37, 40

**Demographics Data**

In [ ]:
demographics = pd.read_csv('data/bio.csv')
demographics = demographics[['subject', 'Age', 'Gender', 'BMI', 'Height ', 'Body weight ', 'Self-identify ', 'A1c PDL (Lab)', 'Fasting GLU - PDL (Lab)', 'Insulin ', 'Triglycerides', 'Cholesterol']]
demographics.columns = ['ID', 'Age', 'Gender', 'BMI', 'Weight', 'Height', 'Race', 'A1c', 'Fasting Glucose', 'Insulin', 'Triglycerides', 'Cholesterol']

# drop those participants identified above that had missing data in the microbiome dataset
demographics = demographics[~demographics['ID'].isin([26, 28, 48])]

In [ ]:
# defining if patient is diabetic, pre-diabetic, or not diabetic based on A1c levels
def diabetic_status(a1c):
    if a1c < 5.7:
        return 'Not Diabetic'
    elif 5.7 <= a1c <= 6.4:
        return 'Pre-Diabetic'
    else:
        return 'Diabetic'

demographics['Diabetic Status'] = demographics['A1c'].apply(diabetic_status)
demographics.head(3)

### Exploratory Data Analysis

In [ ]:
diabetic_counts = demographics['Diabetic Status'].value_counts()

plt.figure(figsize=(8, 6))
diabetic_counts.plot(kind='pie', autopct='%1.1f%%', colors=['darkseagreen', 'lightsteelblue', 'pink'])
plt.title('Distribution of Diabetic Status')
plt.ylabel('')
plt.show()


In [ ]:
print(diabetic_counts)

In [ ]:
gender_counts = demographics['Gender'].value_counts()

plt.figure(figsize=(8, 6))
gender_counts.plot(kind='pie', autopct='%1.1f%%', labels=['Female', 'Male'], colors=['tomato', 'cornflowerblue'])
plt.title('Distribution of Gender')
plt.ylabel('')
plt.show()

In [ ]:
# cleaning up data
def race(value):
    if value == 'Black, African American':
        return 'African American'
    else:
        return value
demographics['Race'] = demographics['Race'].apply(race)

# visualizing distribution
race_counts = demographics['Race'].value_counts()

plt.figure(figsize=(8, 6))
race_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Race')
plt.ylabel('')
plt.show()

In [ ]:
# BMI interpretation (healthy, overweight, obese) based on BMI values - NOTE change to be specific to height/gender
def bmi_category(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Healthy Weight'
    elif 25 <= bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'
demographics['BMI Category'] = demographics['BMI'].apply(bmi_category)


# visualizing distribution
bmi_counts = demographics['BMI Category'].value_counts()

plt.figure(figsize=(8, 6))
bmi_counts.plot(kind='pie', autopct='%1.1f%%' , colors=['olivedrab', 'khaki', 'tan', 'lightyellow'])
plt.title('Distribution of BMI')
plt.ylabel('')
plt.show()

**Participant One Example**

In [ ]:
participant_one = pd.read_csv('participants/CGMacros-001.csv')
participant_one.head(3)

In [ ]:
# transform timestamp column to datetime format
participant_one['Timestamp'] = pd.to_datetime(participant_one['Timestamp'])
# plotting glucose levels 'Libre GL' for participant_one    
plt.figure(figsize=(10, 5))
plt.plot(participant_one['Timestamp'], participant_one['Libre GL'], marker='o')
plt.title('Glucose Levels Over Time for Participant One')
plt.xlabel('Timestamp')
plt.ylabel('Libre GL')
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# plotting glucose data for participant_one only for day one (2020-05-01)
participant_one_day_one = participant_one[participant_one['Timestamp'].dt.date == pd.to_datetime('2020-05-02').date()]
plt.figure(figsize=(10, 5))
plt.plot(participant_one_day_one['Timestamp'], participant_one_day_one['Dexcom GL'], marker='o')
plt.title('Dexcom Readings on May 2nd for Participant One')
plt.xlabel('Timestamp')
plt.ylabel('Dexcom GL (mg/dL)')
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# plotting glucose data for participant_one only for day one (2020-05-01)
participant_one_day_one = participant_one[participant_one['Timestamp'].dt.date == pd.to_datetime('2020-05-02').date()]
plt.figure(figsize=(10, 5))
plt.plot(participant_one_day_one['Timestamp'], participant_one_day_one['Libre GL'], marker='o')
plt.title('Libre Readings on May 2nd for Participant One')
plt.xlabel('Timestamp')
plt.ylabel('Libre GL (mg/dL)')
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

### Random Forest Model

In [ ]:
X = microbiome.drop(columns=['subject', 'Gut Microbiome Health']) # drop id column (don't want to use as feature)
y = demographics['Diabetic Status']      # col we want to predict

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

- precision: when label is predicted, how often is it correct? 
  - minimize false positives
- recall: how many of the positives were actually predicted?
  - minimize false negatives
- f1 score: balance of precision and recall
- support: 

In [ ]:
# confusion matrix - diagonal elements represent correct classifications, off-diagonal elements represent misclassifications
print(confusion_matrix(y_test, y_pred))

In [ ]:
# feature importance (doesn't indicate direction) -> 4 features, lasso selection, mr-mr, pca
# XG boost is most powerful
importances = pd.Series(model.feature_importances_, index=X.columns)
print(importances.sort_values(ascending=False))

In [ ]:
# AUC score 
roc_auc_score(y_test, model.predict_proba(X_test), multi_class="ovr")

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X)